## Домашнее задание:
Для уравнения теплопроводности
$$\frac{\partial u}{\partial t} = \frac{\partial^2 u}{\partial x^2} + \sin(x) + \frac{2t}{t^2+1}, x \in [0,\pi], t \in [0,10]$$
с начальными условиями
$$u(x,0) = \sin(x), x \in [0,\pi]$$
и граничными условиями
$$u(0,t) = \ln(t^2+1), u(\pi,t) = \ln(t^2+1), t \in [0,10]$$

### Задача: запрограммировать схемы: явную, неявную, Кранка–Никольсон.
Базовая сетка с шагами: $h = \pi/5$ и $\tau = 0.25$

### Условия: уменьшить шаги по времени и по пространству. 
Как влияет соотношение шагов на точность решения?  
Начально краевая задача имеет точное решение
$$u(x,t) = \sin(x) + \ln(t^2+1)$$

## Краевые и начальные условия:
Общий вид уравнения теплопроводности:
$$\frac{\partial u}{\partial t} = a^2\frac{\partial^2 u}{\partial x^2} + g(x,t), x \in [0,l], t \in [0,T]$$
начальное условие:
$$u(x,0) = u_0(x), x \in [0,l],$$
левое и правое граничные условия:
$$u(0,t) = \mu_0(t), t \in [0,T]$$
$$u(l,t) = \mu_1(t), t \in [0,T]$$

Для нашей задачи запишем так:  
$$g(x,t)=\sin(x) + \frac{2t}{t^2+1}, x \in [0,l], t \in [0,T]$$
$$u_0(x)=\sin(x), x \in [0,l]$$
$$\mu_0(t)=\ln(t^2+1), t \in [0,T]$$
$$\mu_1(t)=\ln(t^2+1), t \in [0,T]$$
$$a=1, l=\pi, T=10$$

С помощью $u_i^j$ будем обозначать приближённое значение $u(x_i,t_j)$

Сетка $N x M$, с шагом $h = \pi/5$ и $\tau = 0.25$, где $n = \frac{x}{h}$, $m = \frac{T}{\tau} $

In [1]:
import math
import numpy as np

l = math.pi
T = 10
h_base = math.pi / 5
tau_base = 0.25

def g(x,t):
    return math.sin(x) + 2*t/(t*t+1)
g = np.vectorize(g)

def u0(x):
    return math.sin(x)

def mu(t):
    return math.log(t*t+1)

def u_exact(x,t):
    return math.sin(x) + math.log(t*t+1)
u_exact = np.vectorize(u_exact)

def get_x(i,h):
    return i*h
get_x = np.vectorize(get_x)
    
def get_t(j,tau):
    return j*tau
get_t = np.vectorize(get_t)

In [2]:
class FiniteDifferenceMethod:
    def __init__(self,h,tau):
        self.h = h
        self.tau = tau
        xn = int(round(l/h))+1       #кол-во узлов
        tn = int(round(T/tau))+1     #кол-во узлов
        self.grid = np.zeros((xn,tn))
        self.fill()
        
    def fill(self):
        tau = self.tau
        h = self.h
        u = self.grid
        N,M = u.shape
        # Начальные условия:
        for i in range(0,N):
            x = i*h
            u[i,0] = u0(x)
        # Граничные условия:
        for j in range(0,M):
            t = j*tau
            u[0,j] = mu(t)
            u[N-1,j] = mu(t)
    
    def nearest_node(self,x,t):
        i = int(round(x / self.h))
        j = int(round(t / self.tau))
        return i, j
    
    def calc(self,x,t):
        i,j = self.nearest_node(x,t)
        return self.grid[i,j]

# Явная схема
формула:
$$\frac{u_i^{j+1}-u_i^j}{\tau} = a^2\frac{u_{i+1}^j-2u_i^j+u_{i-1}^j}{h^2} + g_i^j$$

$$u_i^{j+1} = \tau a^2\frac{u_{i+1}^j-2u_i^j+u_{i-1}^j}{h^2} + \tau g_i^j + u_i^j$$
выразим:$$u_i^{j+1} =  u_i^j + \tau\frac{u_{i+1}^j-2u_i^j+u_{i-1}^j}{h^2} + \tau\sin(x_i) + \tau\frac{2t_j}{t_j^2+1}$$
Схема - условно устойчива

In [3]:
class ExplicitMethod(FiniteDifferenceMethod):
    def execute(self):
        u = self.grid
        N,M = u.shape
        tau = self.tau
        h = self.h
        
        for j in range(0,M-1):
            t = j*tau
            for i in range(1,N-1):
                x = i*h
                u[i,j+1] = u[i,j] + tau * (u[i+1,j]-2*u[i,j]+u[i-1,j]) / (h**2)  + tau*g(x,t)
        return self

# Неявная схема
формула:
$$\frac{u_i^{j+1}-u_i^j}{\tau} = a^2\frac{u_{i+1}^{j+1}-2u_i^{j+1}+u_{i-1}^{j+1}}{h^2} + g_i^j$$


Составим следующую систему: 


$$u_0^{j+1} = u_N^{j+1} = \ln(t_{j+1}^2+1), \; j = 0,\dots, M-1 $$

$$u_i^0 = \sin(x_i), \; i = 1,\dots,N-1 $$

$$  \frac{\tau}{h^2}u_{i-1}^{j+1}-\big(1+\frac{2\tau}{h^2}\big)u_i^{j+1}+\frac{\tau}{h^2}u_{i+1}^{j+1} = -\big(u_i^j+\tau\sin(x_i) + \tau\frac{2t_j}{t_j^2+1}\big), \; i = 1,\dots,N-1 $$



Получим трёхдиагональную матрицу для каждого $ j = 0,\dots, M-1 $ 


$$\left(
\begin{array}{cccccccc|c}
    1 & 0 & 0 & 0 & \dots & 0 & 0 & 0 & \ln(t_{j+1}^2+1) \\
    \frac{\tau}{h^2} & -(1+\frac{2\tau}{h^2}) & \frac{\tau}{h^2} & 0 & \dots & 0 & 0 & 0 & -\big(u_1^j+\tau\sin(x_1) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    0 & \frac{\tau}{h^2} & -(1+\frac{2\tau}{h^2}) & \frac{\tau}{h^2} & \dots & 0 & 0 & 0 & -\big(u_2^j+\tau\sin(x_2) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \dots & \frac{\tau}{h^2} & 0 & 0 & -\big(u_{N-3}^j+\tau\sin(x_{N-3}) + \tau\frac{2t_j}{t_j^2+1}\big) \\\\
    0 & 0 & 0 & 0 & \dots & -(1+\frac{2\tau}{h^2}) & \frac{\tau}{h^2} & 0 & -\big(u_{N-2}^j+\tau\sin(x_{N-2}) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    0 & 0 & 0 & 0 & \dots & \frac{\tau}{h^2} & -(1+\frac{2\tau}{h^2}) & \frac{\tau}{h^2} & -\big(u_{N-1}^j+\tau\sin(x_{N-1}) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    0 & 0 & 0 & 0 & \dots & 0 & 0 & 1 & \ln(t_{j+1}^2+1)
    \end{array}
\right)$$

Для каждого $j$ система решается методом прогонки, так как имеет трёхдиагональную матрицу.

Схема абсолютно устойчива

In [4]:
from scipy.linalg import solve_banded

class ImplicitMethod(FiniteDifferenceMethod):
    def execute(self):
        h = self.h
        tau = self.tau
        u = self.grid
        N,M = u.shape
        N -= 1
        M -= 1

        ud = np.insert(np.repeat(tau/(h**2), N-1), [0,0], 0)
        d = np.insert(np.repeat(-(1+2*tau/(h**2)), N-1), [0,N-1], 1)
        ld = np.insert(np.repeat(tau/(h**2), N-1), [N-1,N-1], 0)
        xs = np.arange(1,N)*h
                
        for j in range(0,M):
            t = j*tau
            t1 = (j+1)*tau
            r = -(u[1:N,j] + tau*g(xs,t1))
            r = np.insert(r, 0, mu(t1))
            r = np.insert(r, N, mu(t1))
            u[:,j+1] = solve_banded((1, 1), np.matrix([ud, d, ld]), r)    #применяем прогонку
        return self

# Схема Кранка–Никольсон
имеет вид:
$$\frac{u_i^{j+1}-u_i^j}{\tau} = \frac{a^2}{2}\big(\frac{u_{i+1}^j-2u_i^j+u_{i-1}^j}{h^2} + \frac{u_{i+1}^{j+1}-2u_i^{j+1}+u_{i-1}^{j+1}}{h^2}\big) + g_i^j$$


Составим систему

  $$ u_0^{j+1} = u_N^{j+1} = \ln(t_{j+1}^2+1), \;  j = 0,\dots, M-1 $$
  $$ \frac{\tau}{2h^2}u_{i-1}^{j+1}-\big(1+\frac{\tau}{h^2}\big)u_i^{j+1}+\frac{\tau}{2h^2}u_{i+1}^{j+1} = -\big(\frac{\tau}{2h^2}u_{i-1}^j+\big(1-\frac{\tau}{h^2}\big)u_i^j+\frac{\tau}{2h^2}u_{i+1}^j + \tau\sin(x_i) + \tau\frac{2t_j}{t_j^2+1}\big), \; i = 1,\dots,N-1 $$
  $$ u_i^0 = \sin(x_i), \; i = 1,\dots,N-1$$

Получим такую трёхдиагональную матрицу

$$\left(
\begin{array}{cccccccc|c}
    1 & 0 & 0 & 0 & \dots & 0 & 0 & 0 & \ln(t_{j+1}^2+1) \\
    \frac{\tau}{2h^2} & -(1+\frac{\tau}{h^2}) & \frac{\tau}{2h^2} & 0 & \dots & 0 & 0 & 0 & -\big(\frac{\tau}{2h^2}u_0^j+\big(1-\frac{\tau}{h^2}\big)u_1^j+\frac{\tau}{2h^2}u_2^j + \tau\sin(x_1) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    0 & \frac{\tau}{2h^2} & -(1+\frac{\tau}{h^2}) & \frac{\tau}{2h^2} & \dots & 0 & 0 & 0 & -\big(\frac{\tau}{2h^2}u_1^j+\big(1-\frac{\tau}{h^2}\big)u_2^j+\frac{\tau}{2h^2}u_3^j + \tau\sin(x_2) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \dots & \frac{\tau}{2h^2} & 0 & 0 & -\big(\frac{\tau}{2h^2}u_{N-4}^j+\big(1-\frac{\tau}{h^2}\big)u_{N-3}^j+\frac{\tau}{2h^2}u_{N-2}^j + \tau\sin(x_{N-3}) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    0 & 0 & 0 & 0 & \dots & -(1+\frac{\tau}{h^2}) & \frac{\tau}{2h^2} & 0 & -\big(\frac{\tau}{2h^2}u_{N-3}^j+\big(1-\frac{\tau}{h^2}\big)u_{N-2}^j+\frac{\tau}{2h^2}u_{N-1}^j + \tau\sin(x_{N-2}) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    0 & 0 & 0 & 0 & \dots & \frac{\tau}{2h^2} & -(1+\frac{\tau}{h^2}) & \frac{\tau}{2h^2} & -\big(\frac{\tau}{2h^2}u_{N-2}^j+\big(1-\frac{\tau}{h^2}\big)u_{N-1}^j+\frac{\tau}{2h^2}u_N^j + \tau\sin(x_{N-1}) + \tau\frac{2t_j}{t_j^2+1}\big) \\
    0 & 0 & 0 & 0 & \dots & 0 & 0 & 1 & \ln(t_{j+1}^2+1)
    \end{array}
\right)$$


Аналогично неявному методу, для каждого $j$ система решается методом прогонки

Схема безусловно устойчива при любом соотношении s и h

In [5]:
class CrankNicolsonMethod(FiniteDifferenceMethod):
    def execute(self):
        h = self.h
        tau = self.tau
        u = self.grid
        N,M = u.shape
        N -= 1
        M -= 1

        ud = np.insert(np.repeat(tau/(2*h**2), N-1), [0,0], 0)
        d = np.insert(np.repeat(-(1+tau/(h**2)), N-1), [0,N-1], 1)
        ld = np.insert(np.repeat(tau/(2*h**2), N-1), [N-1,N-1], 0)
        
        xs = np.arange(1,N)*h
        for j in range(0,M):
            t = j*tau
            t1 = (j+1)*tau
            r = -(tau/(2*h**2)*u[0:N-1,j] + (1-tau/(h**2))*u[1:N,j] + tau/(2*h**2)*u[2:N+1,j] + tau*g(xs,t1))
            r = np.insert(r, 0, mu(t1))
            r = np.insert(r, N, mu(t1))
            u[:,j+1] = solve_banded((1, 1), np.matrix([ud, d, ld]), r)    #делаем прогонку
        return self

# Оценка погрешности

Оценим максимум модуля погрешности (разности между точным и приближенным значениями) для различных значений $h$ и $\tau$

In [6]:
def norma(grid_function):
    return np.max(np.abs(grid_function))

def eps(f_get_approximate, f_exact, h, tau):
    return f_get_approximate - f_exact

def get_approximate_error(method, h, tau):
    #вычислили значения приближенной функции
    f_get_approximate = method(h, tau).execute().grid    
    
    #вычисляем точные значения 
    xn = int(round(l/h))+1
    tn = int(round(T/tau))+1
    xs = np.tile(get_x(np.arange(0,xn),h), (tn, 1)).T
    ts = np.tile(get_t(np.arange(0,tn),tau), (xn, 1))
    f_exact = u_exact(xs, ts)   
    
    #вычисляем норму ошибки
    return norma(eps(f_get_approximate, f_exact, h, tau))

In [7]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.display import display

def evaluate_quality(method):
    hn = 5
    taun = 6
    results = pd.DataFrame(columns=['h', 'tau', 'error'])
    
    h_array = h_base / np.arange(1,hn+1)
    tau_array = 0.5 / 2 ** np.arange(1,taun+1)
    
    for h in h_array :
        for tau in tau_array :
            error = get_approximate_error(method, h, tau)
            results = results.append({'h': h, 'tau': tau, 'error': error}, ignore_index=True)
    
    display(results)
    plt.show() 

### Схема Кранка–Никольсон

In [8]:
cn_results = evaluate_quality(CrankNicolsonMethod)

,h,tau,error
0,0.628319,0.250000,0.099511
1,0.628319,0.125000,0.059343
2,0.628319,0.062500,0.038705
3,0.628319,0.031250,0.031445
4,0.628319,0.015625,0.031680
5,0.628319,0.007812,0.031797
6,0.314159,0.250000,0.092368
7,0.314159,0.125000,0.049952
8,0.314159,0.062500,0.027499
9,0.314159,0.031250,0.015989


### Неявный метод

In [11]:
impl_results = evaluate_quality(ImplicitMethod)

,h,tau,error
0,0.628319,0.250000,0.091947
1,0.628319,0.125000,0.057017
2,0.628319,0.062500,0.038009
3,0.628319,0.031250,0.031443
4,0.628319,0.015625,0.031679
5,0.628319,0.007812,0.031796
6,0.314159,0.250000,0.084690
7,0.314159,0.125000,0.047654
8,0.314159,0.062500,0.026877
9,0.314159,0.031250,0.015816


### Явный метод

In [10]:
expl_results = evaluate_quality(ExplicitMethod)

,h,tau,error
0,0.628319,0.250000,9.192398e-02
1,0.628319,0.125000,4.020205e-02
2,0.628319,0.062500,3.564609e-02
3,0.628319,0.031250,3.351771e-02
4,0.628319,0.015625,3.255921e-02
5,0.628319,0.007812,3.215981e-02
6,0.314159,0.250000,1.752724e+34
7,0.314159,0.125000,4.576176e+43
8,0.314159,0.062500,3.315546e+22
9,0.314159,0.031250,1.038507e-02


# Выводы

Результаты эксперимента в results.xslx. Для быстрого просмотра в result.PNG

Явный метод при малых шагах по времени и пространству результаты ухудшаются из-за накопления ошибок округления. Так как метод условно устойчив: в первом столбце и последней строке таблицы (выделены зеленым цветом), наблюдается снижение ошибки при уменьшении шагов по времени или пространству.

Схема Кранка–Никольсон и неявный метод обладают абсолютной устойчивостю, при изменении шага дают лучшие результаты.